# Pobieranie obrazów satelitarnych

## Import bibliotek

In [ ]:
from planet import Planet, data_filter, order_request
from collections import defaultdict
from datetime import datetime, timedelta
from itertools import islice
from collections import defaultdict
from datetime import datetime
    
import json
import os

pl = Planet()

## Ustawienie obszaru

In [ ]:
remedhus_aoi = {
    "type": "Polygon",
    "coordinates": [[
        [-5.7, 41.1],  # SW corner
        [-5.1, 41.1],  # SE corner
        [-5.1, 41.5],  # NE corner
        [-5.7, 41.5],  # NW corner
        [-5.7, 41.1]   # zamknięcie poligonu
    ]]
}

## Wyszukiwanie obrazów

In [ ]:
def search_daily_imagery(start_date, end_date, aoi, max_cloud_cover=0.2):
    if isinstance(start_date, str):
        start_date = datetime.fromisoformat(start_date)
    if isinstance(end_date, str):
        end_date = datetime.fromisoformat(end_date)
    
    search_filter = data_filter.and_filter([
        data_filter.geometry_filter(aoi),
        data_filter.date_range_filter(
            field_name="acquired",
            gte=start_date,
            lte=end_date
        ),
        data_filter.range_filter(
            field_name="cloud_cover",
            gte=0.0,
            lte=max_cloud_cover
        ),
        data_filter.string_in_filter(
            field_name="quality_category",
            values=["standard"]
        )
    ])
    
    results = pl.data.search(
        item_types=["PSScene"],
        search_filter=search_filter
    )
    
    return results


In [ ]:
def print_search_summary(items):
    if not items:
        print("Brak znalezionych obrazów")
        return
    
    # Grupowanie po datach
    images_by_date = defaultdict(int)
    cloud_covers = []
    
    for item in items:
        date = item['properties']['acquired'][:10]
        images_by_date[date] += 1
        cloud_covers.append(item['properties']['cloud_cover'])
    
    print(f"\n=== Podsumowanie wyszukiwania ===")
    print(f"Łączna liczba obrazów: {len(items)}")
    print(f"Pokrycie czasowe: {len(images_by_date)} dni")
    print(f"Średnie zachmurzenie: {sum(cloud_covers)/len(cloud_covers)*100:.1f}%")
    print(f"Min zachmurzenie: {min(cloud_covers)*100:.1f}%")
    print(f"Max zachmurzenie: {max(cloud_covers)*100:.1f}%")
    
    # Pokaż pierwsze 5 dni
    sorted_dates = sorted(images_by_date.keys())[:5]
    print(f"\nPrzykładowe dni:")
    for date in sorted_dates:
        print(f"  {date}: {images_by_date[date]} obraz(ów)")


start = "2024-12-01T00:00:00Z"
end = "2024-12-31T23:59:59Z"
results = search_daily_imagery(start, end, remedhus_aoi, max_cloud_cover=0.15)
items = list(islice(results, 1000))
print_search_summary(items)




=== Podsumowanie wyszukiwania ===
Łączna liczba obrazów: 80
Pokrycie czasowe: 14 dni
Średnie zachmurzenie: 2.7%
Min zachmurzenie: 0.0%
Max zachmurzenie: 15.0%

Przykładowe dni:
  2024-12-04: 8 obraz(ów)
  2024-12-05: 10 obraz(ów)
  2024-12-07: 3 obraz(ów)
  2024-12-08: 4 obraz(ów)
  2024-12-09: 10 obraz(ów)


## Tworzenie zamówienia

In [ ]:
def create_order_for_items(item_ids, order_name="REMEDHUS_daily"):
    # Wybór typu produktu
    # ortho_analytic_8b_sr - 8-kanałowe dane z korekcją atmosferyczną
    # ortho_analytic_4b_sr - 4-kanałowe dane z korekcją atmosferyczną
    
    request = order_request.build_request(
        name=order_name,
        products=[
            order_request.product(
                item_ids=item_ids,
                product_bundle='analytic_8b_sr_udm2',  # 8-band surface reflectance
                item_type='PSScene'
            )
        ],
        tools=[
            order_request.clip_tool(aoi=remedhus_aoi),  # przycięcie do AOI
            order_request.reproject_tool(
                projection="EPSG:4326",  # WGS84
                kernel="cubic"
            )
        ]
    )
    
    # Utworzenie zamówienia
    order = pl.orders.create_order(request)
    print(f"Zamówienie utworzone: {order['id']}")
    
    return order

# Przykład: wybierz pierwsze 10 obrazów
item_ids = [item['id'] for item in items]
order = create_order_for_items(item_ids)

Zamówienie utworzone: ad8202e7-1f0c-4380-b786-b6bee3c48096


## Pobieranie zamówienia

In [ ]:
def download_order(order_id, dest_dir):
    # Oczekiwanie na przetworzenie
    print("Oczekiwanie na przetworzenie zamówienia...")
    pl.orders.wait(order_id, max_attempts=300, delay=30)
    
    # Pobieranie
    print(f"Pobieranie do {dest_dir}...")
    os.makedirs(dest_dir, exist_ok=True)
    pl.orders.download_order(order_id, directory=dest_dir, overwrite=True)
    
    print("Pobieranie zakończone")

# Pobierz zamówienie
download_order(order['id'], dest_dir='./data/remedhus_satimg')

Oczekiwanie na przetworzenie zamówienia...
Pobieranie do ./data/remedhus_2024...
Pobieranie zakończone!


## Pobieranie serii czasowej

In [ ]:
def download_daily_timeseries_advanced(start_date, end_date, aoi, output_dir, max_images=None, batch_size=500, resume=False):
    
    os.makedirs(output_dir, exist_ok=True)
    
    # Plik z progressem
    progress_file = os.path.join(output_dir, 'download_progress.json')
    
    # Wczytaj poprzedni progress (jeśli resume=True)
    downloaded_batches = set()
    if resume and os.path.exists(progress_file):
        with open(progress_file, 'r') as f:
            progress = json.load(f)
            downloaded_batches = set(progress.get('downloaded_batches', []))
        print(f"📂 Wznowienie pobierania - {len(downloaded_batches)} partii już pobrano")
    
    # Wyszukaj obrazy
    print(f"\n🔍 Wyszukiwanie obrazów od {start_date} do {end_date}...")
    results = search_daily_imagery(start_date, end_date, aoi)
    
    # Konwersja do listy z limitem
    print("⏳ Konwersja wyników...")
    if max_images:
        items = list(islice(results, max_images))
    else:
        items = list(islice(results, 10000))  # Safety limit
    
    print(f"✓ Znaleziono {len(items)} obrazów\n")
    
    if len(items) == 0:
        print("⚠️  Brak obrazów - sprawdź parametry wyszukiwania")
        return {'status': 'no_images', 'count': 0}
    
    # Analiza pokrycia czasowego
    images_by_date = defaultdict(list)
    for item in items:
        date = item['properties']['acquired']
        images_by_date[date].append(item['id'])
    
    print(f"📅 Pokrycie czasowe:")
    print(f"   Dni z obrazami: {len(images_by_date)}")
    print(f"   Pierwsza data: {min(images_by_date.keys())}")
    print(f"   Ostatnia data: {max(images_by_date.keys())}")
    print(f"   Średnia obrazów/dzień: {len(items)/len(images_by_date):.1f}")
    
    # Przygotuj partie
    all_ids = [item['id'] for item in items]
    num_batches = (len(all_ids) + batch_size - 1) // batch_size
    
    print(f"\n📦 Pobieranie:")
    print(f"   Łączna liczba obrazów: {len(all_ids)}")
    print(f"   Rozmiar partii: {batch_size}")
    print(f"   Liczba partii: {num_batches}\n")
    
    # Statystyki
    stats = {
        'start_time': datetime.now().isoformat(),
        'total_images': len(all_ids),
        'num_batches': num_batches,
        'successful_batches': 0,
        'failed_batches': 0,
        'downloaded_batches': list(downloaded_batches)
    }
    
    # Pobieraj partie
    for i in range(0, len(all_ids), batch_size):
        batch_num = i // batch_size + 1
        
        # Pomiń jeśli już pobrano
        if batch_num in downloaded_batches:
            print(f"⏭️  Partia {batch_num}/{num_batches} - już pobrana, pomijam")
            continue
        
        batch = all_ids[i:i+batch_size]
        
        print(f"\n{'='*70}")
        print(f"📥 Partia {batch_num}/{num_batches} ({len(batch)} obrazów)")
        print(f"{'='*70}")
        
        try:
            # Utwórz zamówienie
            order = create_order_for_items(
                batch,
                order_name=f"REMEDHUS_batch_{batch_num:03d}"
            )
            
            # Pobierz
            batch_dir = os.path.join(output_dir, f'batch_{batch_num:03d}')
            download_order(order['id'], dest_dir=batch_dir)
            
            # Zaktualizuj progress
            downloaded_batches.add(batch_num)
            stats['successful_batches'] += 1
            stats['downloaded_batches'] = list(downloaded_batches)
            
            # Zapisz progress
            with open(progress_file, 'w') as f:
                json.dump(stats, f, indent=2)
            
            print(f"✅ Partia {batch_num} pobrana pomyślnie!")
            
        except KeyboardInterrupt:
            print(f"\n\n⚠️  Przerwano przez użytkownika (Ctrl+C)")
            print(f"📊 Postęp: {stats['successful_batches']}/{num_batches} partii")
            print(f"💾 Progress zapisany w: {progress_file}")
            print(f"🔄 Wznów pobieranie używając resume=True")
            break
            
        except Exception as e:
            print(f"❌ Błąd przy partii {batch_num}: {e}")
            print(f"   Kontynuuję z następną partią...")
            continue
    
    # Finalizacja
    stats['end_time'] = datetime.now().isoformat()
    stats['completion_rate'] = stats['successful_batches'] / num_batches * 100
    
    with open(progress_file, 'w') as f:
        json.dump(stats, f, indent=2)
    
    # Podsumowanie
    print(f"\n{'='*70}")
    print(f"✅ POBIERANIE ZAKOŃCZONE")
    print(f"{'='*70}")
    print(f"📊 Statystyki:")
    print(f"   Pomyślne partie: {stats['successful_batches']}/{num_batches}")
    print(f"   Nieudane partie: {stats['failed_batches']}/{num_batches}")
    print(f"   Wskaźnik sukcesu: {stats['completion_rate']:.1f}%")
    print(f"   Katalog: {output_dir}")
    print(f"{'='*70}\n")
    
    return stats


stats = download_daily_timeseries_advanced(
    start_date="2024-12-01",
    end_date="2024-12-31",
    aoi=remedhus_aoi,
    output_dir="./data/remedhus_timeseries",
    max_images=1000,  # Limit dla testów
    batch_size=500
)


🔍 Wyszukiwanie obrazów od 2024-12-01 do 2024-12-31...
⏳ Konwersja wyników...
✓ Znaleziono 84 obrazów

📅 Pokrycie czasowe:
   Dni z obrazami: 15
   Pierwsza data: 2024-12-01
   Ostatnia data: 2024-12-23
   Średnia obrazów/dzień: 5.6

📦 Pobieranie:
   Łączna liczba obrazów: 84
   Rozmiar partii: 500
   Liczba partii: 1


📥 Partia 1/1 (84 obrazów)
Zamówienie utworzone: c7a98ddf-2e17-49c5-96f7-c6491c2b2a16
Oczekiwanie na przetworzenie zamówienia...
Pobieranie do ./data/remedhus_timeseries/batch_001...
Pobieranie zakończone!
✅ Partia 1 pobrana pomyślnie!

✅ POBIERANIE ZAKOŃCZONE
📊 Statystyki:
   Pomyślne partie: 1/1
   Nieudane partie: 0/1
   Wskaźnik sukcesu: 100.0%
   Katalog: ./data/remedhus_timeseries

